In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
import os
import shutil

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

In [0]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *

Using TensorFlow backend.


## Dataset Pipeline

In [0]:
# !wget https://he-public-data.s3-ap-southeast-1.amazonaws.com/HE_Challenge_data.zip

In [0]:
# !unzip HE_Challenge_data

In [0]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [0]:
train = train.values

In [0]:
folders, num_samples = np.unique(train[:, 1], return_counts=True)

In [0]:
# make subdirectory of each class
for folder in folders:
  p = os.path.join('data/train', str(folder))
  if not os.path.isdir(p):
    os.mkdir(p)


In [0]:
for i in range(train.shape[0]):
  dest = os.path.join('data/train/', str(train[i][1]))
  src = os.path.join('data/train', str(train[i][0]) + '.jpg')
  shutil.move(src, dest)

#### training-validation split

In [0]:
SPLIT = 0.95 # 95% for training 5% for validation

In [0]:


# create the validation directory
if not os.path.isdir('data/val'):
  os.mkdir('data/val')



In [0]:
# creating subdirectories (of each category) in validation directory
for folder in folders:
  p = os.path.join('data/val', str(folder))
  if not os.path.isdir(p):
    os.mkdir(p)


In [0]:
# move the 3% data from training directory to validation directory
for f in folders:
  path = 'data/train/'+str(f)
  imgs = os.listdir(path)
  split_size = int(SPLIT*len(imgs))
  file_to_move = imgs[split_size:]
  
  for img_f in file_to_move:
    src = os.path.join(path, img_f)
    dest = os.path.join('data/val/'+str(f), img_f)
    shutil.move(src, dest)

### Data Augmentation


In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 60,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
)
val_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
)


In [0]:
train_gen = train_img_gen.flow_from_directory(
    'data/train',
    #target_size = (224, 224),
    batch_size = 8,
    

)
val_gen = val_img_gen.flow_from_directory(
    'data/val/',
    #target_size = (224, 224),
    batch_size = 8
)

Found 17562 images belonging to 102 classes.
Found 978 images belonging to 102 classes.


In [0]:
for (x, y) in train_gen:
  print(x.shape, y.shape)
  break

(8, 256, 256, 3) (8, 102)


### Inception Network architecture

In [0]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, name=None):
  
  # 1x1 convolution
  conv_1x1 = Conv2D(filters_1x1, (1, 1), padding = 'same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  
  # 3x3 convolution 1x1 convolution to reduce the computation
  conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
  
  # 5x5 convolution 1x1 convolution to reduce the computation
  conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
  
  # max pooling 1x1 convolution to reduce the computation
  pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
  
  # concatenate em' all
  output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
  
  return output
   
  
  
  

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [0]:
input_layer = Input(shape=(256, 256, 3))

In [0]:
model = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu', name='conv_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)

In [0]:
model = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='maxpool_3x3/2')(model)

In [0]:
model = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(model)

In [0]:
model = Conv2D(64, (1, 1), strides=(1, 1), padding='same', activation='relu', name='conv_1x1/3', kernel_initializer=kernel_init, bias_initializer=bias_init)(model)

In [0]:
model = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv3x3/1')(model)

In [0]:
model = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(model)

In [0]:
model = inception_module(model,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

model = inception_module(model,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

model = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(model)

model = inception_module(model,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


m1 = AveragePooling2D((5, 5), strides=3)(model)
m1 = Conv2D(128, (1, 1), padding='same', activation='relu')(m1)
m1 = Flatten()(m1)
m1 = Dense(1024, activation='relu')(m1)
m1 = Dropout(0.25)(m1)
m1 = Dense(102, activation='softmax', name='auxilliary_output_1')(m1)

model = inception_module(model,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

model = inception_module(model,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

model = inception_module(model,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


m2 = AveragePooling2D((5, 5), strides=3)(model)
m2 = Conv2D(128, (1, 1), padding='same', activation='relu')(m2)
m2 = Flatten()(m2)
m2 = Dense(1024, activation='relu')(m2)
m2 = Dropout(0.25)(m2)
m2 = Dense(102, activation='softmax', name='auxilliary_output_2')(m2)

model = inception_module(model,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

model = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(model)

model = inception_module(model,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

model = inception_module(model,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

model = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(model)

model = Dropout(0.25)(model)

model = Dense(102, activation='softmax', name='output')(model)

In [0]:
model = Model(input_layer, outputs = model, name='inception_v1')

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv_7x7/2 (Conv2D)             (None, 128, 128, 64) 9472        input_7[0][0]                    
__________________________________________________________________________________________________
maxpool_3x3/2 (MaxPooling2D)    (None, 64, 64, 64)   0           conv_7x7/2[0][0]                 
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 64, 64, 64)   256         maxpool_3x3/2[0][0]              
__________________________________________________________________________________________________
conv_1x1/3

## Training Inception Net

In [0]:
# we will train two other models 
model_1 = model
model_2 = model

In [0]:
es = keras.callbacks.EarlyStopping(monitor ='val_acc', min_delta=0.5, patience=5)

In [0]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [0]:
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

In [0]:
lr_sc = LearningRateScheduler(decay, verbose=1)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
model_cp_0 = keras.callbacks.ModelCheckpoint('inception_v1.h5', monitor='val_acc', save_best_only=True, save_weights_only=True)

In [0]:
model.fit_generator(train_gen, steps_per_epoch=17562//8, validation_data=val_gen, validation_steps=978//2, epochs=epochs, callbacks=[ model_cp_0])

Epoch 1/25
2195/2195 [==============================] - 363s 165ms/step - loss: 1.9979 - acc: 0.4490 - val_loss: 1.7780 - val_acc: 0.5008
Epoch 2/25
2195/2195 [==============================] - 363s 165ms/step - loss: 1.9219 - acc: 0.4678 - val_loss: 1.8167 - val_acc: 0.5013
Epoch 3/25
2195/2195 [==============================] - 364s 166ms/step - loss: 1.7961 - acc: 0.4989 - val_loss: 1.6213 - val_acc: 0.5556
Epoch 4/25
2195/2195 [==============================] - 363s 166ms/step - loss: 1.7514 - acc: 0.5128 - val_loss: 1.6076 - val_acc: 0.5633
Epoch 5/25
2195/2195 [==============================] - 363s 165ms/step - loss: 1.6498 - acc: 0.5367 - val_loss: 1.6675 - val_acc: 0.5409
Epoch 6/25
2195/2195 [==============================] - 364s 166ms/step - loss: 1.6120 - acc: 0.5494 - val_loss: 1.4415 - val_acc: 0.5970
Epoch 7/25
2195/2195 [==============================] - 363s 165ms/step - loss: 1.5054 - acc: 0.5687 - val_loss: 1.3247 - val_acc: 0.6296
Epoch 8/25
2195/2195 [============

In [0]:
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.11.0-cp36-none-any.whl size=14469 sha256=03bd8dd1491ef6cb5749d123faf2bcc220939ce2fc0643ade5531da22f2c444a
  Stored in directory: /root/.cache/pip/wheels/31/ee/eb/6c72886931d3e2dfba3c701721297daf8afabc740b742b2555
Successfully built keras-radam
